# Sintetic Software Used  To Analysis of Cassandra's Performance

## Imports

In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import time
import matplotlib.pyplot as plt

## Database configuration

### Database Structure
<pre>+-----------------------+ 
|        table          |
+-----------------------+
| A0 | UUID        |    |
| A1 | INT         | PK |
| A2 | BIGINT      |    |
| A3 | DOUBLE      |    |
| A4 | VARCHAR(26) |    |
| A5 | TEXT        |    |
| A6 | DATE        |    |
| A7 | TIMESTAMP   |    |
+-----------------------+</pre>

In [2]:
cluster = Cluster(['localhost'], auth_provider=PlainTextAuthProvider('cassandra', 'cassandra'))
session = cluster.connect()

# Set User
# session.execute('ALTER USER cassandra WITH PASSWORD \'cassandra\'')

# Create KeySpace
session.execute('CREATE KEYSPACE IF NOT EXISTS analysis WITH replication = {\'class\': \'SimpleStrategy\', \'replication_factor\': 1}')

# Use KeySpace created
session.set_keyspace('analysis')

# Create Table
session.execute("CREATE TABLE IF NOT EXISTS \"table\" ("+
                "A0 uuid, "+
                "A1 int PRIMARY KEY, "+
                "A2 bigint, "+
                "A3 double, "+
                "A4 varchar, "+
                "A5 text, "+
                "A6 date, "+
                "A7 timestamp )")
rows = session.execute('select * from system_schema.columns')
print('+----------------------------------+')
print('|              table               |')
print('+----------------------------------+')
for row in rows:
    if row.table_name == 'table':
        print('|', row.column_name, '|', row.type, ' '*(9-len(row.type)), '|',  row.kind, ' '*(13-len(row.kind)), '|')
print('+----------------------------------+')

+----------------------------------+
|              table               |
+----------------------------------+
| a0 | uuid       | regular        |
| a1 | int        | partition_key  |
| a2 | bigint     | regular        |
| a3 | double     | regular        |
| a4 | text       | regular        |
| a5 | text       | regular        |
| a6 | date       | regular        |
| a7 | timestamp  | regular        |
+----------------------------------+


## Meta data

* R is the number of iterations <br />
* BI is the size of a block of insertions<br />
* BU is the size of a block of updates<br />
* BS is the size of a block of selections<br />

In [ ]:
R = 60
BI = BU = BS = 500000
categories = [i+1 for i in range(R)]

## Definition of functions

In [4]:
def insert(control):
    initialTime = time.time()
    for i in range(BI):
        session.execute('INSERT INTO "table" (A0, A1, A2, A3, A4, A5, A6, A7) VALUES ('+
                        'uuid(), '+
                        str(control)+', '+
                        '9223372036854775807, '+
                        '1.2, '+
                        '\'abcdefghijklmnopqrstuvwxy\', '+
                        '\'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer ultricies lorem metus, vel finibus risus convallis sit amet. Nam auctor ex et ipsum euismod, vel consectetur eros blandit. Sed sit amet enim vitae nisi varius molestie. Maecenas in tortor sem. Ut sit amet lobortis erat, ac egestas libero. Nunc id purus sodales, dictum massa gravida, condimentum diam. Maecenas eu vulputate nunc, vitae tempor odio. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Duis vitae turpis quam. Maecenas at dui at justo vehicula scelerisque eu in eros. Sed volutpat, magna nec pretium tincidunt, risus nibh posuere mauris, quis feugiat augue lorem ut nunc. Lorem ipsum dolor sit amet, consectetur adipiscing elit. In eu tellus nec nulla ultricies efficitur. Interdum et malesuada fames ac ante ipsum primis in faucibus. \', '+
                        'toDate(dateof(now())), '+
                        'dateof(now())'+
                        ')')
        control += 1
    return (time.time() - initialTime), control

In [5]:
def update(control):
    initialTime = time.time()
    for i in range(BI):
        session.execute('UPDATE "table" SET '+
                        'A2 = 9223372036854775800, '+
                        'A3 = 1.3, '+
                        'A4 = \'abbdefghijklmnopqrstuvwxy\', '+
                        'A5 = \'Lorem ipsum dolor site amet, consectetur adipiscing elit. Integer ultricies lorem metus, vel finibus risus convallis sit amet. Nam auctor ex et ipsum euismod, vel consectetur eros blandit. Sed sit amet enim vitae nisi varius molestie. Maecenas in tortor sem. Ut sit amet lobortis erat, ac egestas libero. Nunc id purus sodales, dictum massa gravida, condimentum diam. Maecenas eu vulputate nunc, vitae tempor odio. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Duis vitae turpis quam. Maecenas at dui at justo vehicula scelerisque eu in eros. Sed volutpat, magna nec pretium tincidunt, risus nibh posuere mauris, quis feugiat augue lorem ut nunc. Lorem ipsum dolor sit amet, consectetur adipiscing elit. In eu tellus nec nulla ultricies efficitur. Interdum et malesuada fames ac ante ipsum primis in faucibus. \', '+
                        'A6 = toDate(dateof(now())), '+
                        'A7 = dateof(now())'+
                        'WHERE A1 = '+str(control))
        control += 1
    return (time.time() - initialTime), control

In [6]:
def select(control):
    initialTime = time.time()
    for i in range(BI):
        session.execute('SELECT * FROM "table" WHERE A1 = '+str(control))
        control += 1
    return (time.time() - initialTime), control

## Storage

In [7]:
insertTimes = []
updateTimes = []
selectTimes = []

## DO IT

In [8]:
control = 1
for j in range(R):
    control = BI*j + 1
    t, control = insert(control)
    print('Insert', control)
    insertTimes.append(t)
    control = BI*j + 1
    t, control = update(control)
    print('Update', control)
    updateTimes.append(t)
    control = BI*j + 1
    t, control = select(control)
    print('Select', control)
    selectTimes.append(t)

Insert [374.0993812084198]
Insert [374.0993812084198, 354.6427631378174]
Insert [374.0993812084198, 354.6427631378174, 361.3451943397522]
Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688]
Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688, 357.24955701828003]
Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688, 357.24955701828003, 358.9602324962616]
Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688, 357.24955701828003, 358.9602324962616, 358.5004668235779]
Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688, 357.24955701828003, 358.9602324962616, 358.5004668235779, 358.01245880126953]
Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688, 357.24955701828003, 358.9602324962616, 358.5004668235779, 358.01245880126953, 358.2048010826111]
Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 3

Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688, 357.24955701828003, 358.9602324962616, 358.5004668235779, 358.01245880126953, 358.2048010826111, 353.1637215614319, 357.5835726261139, 359.3443167209625, 347.3865683078766, 352.90055751800537, 353.96637988090515, 354.0449049472809, 354.72680401802063, 354.8077161312103, 350.10274028778076, 354.1178276538849, 353.5008897781372, 355.3994915485382, 353.38027358055115, 354.62901973724365, 353.0570468902588, 353.48124504089355, 353.03538751602173, 349.43913555145264, 354.10142970085144, 353.2013273239136]
Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688, 357.24955701828003, 358.9602324962616, 358.5004668235779, 358.01245880126953, 358.2048010826111, 353.1637215614319, 357.5835726261139, 359.3443167209625, 347.3865683078766, 352.90055751800537, 353.96637988090515, 354.0449049472809, 354.72680401802063, 354.8077161312103, 350.10274028778076, 354.1178276538849, 353.500889778137

Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688, 357.24955701828003, 358.9602324962616, 358.5004668235779, 358.01245880126953, 358.2048010826111, 353.1637215614319, 357.5835726261139, 359.3443167209625, 347.3865683078766, 352.90055751800537, 353.96637988090515, 354.0449049472809, 354.72680401802063, 354.8077161312103, 350.10274028778076, 354.1178276538849, 353.5008897781372, 355.3994915485382, 353.38027358055115, 354.62901973724365, 353.0570468902588, 353.48124504089355, 353.03538751602173, 349.43913555145264, 354.10142970085144, 353.2013273239136, 354.0940809249878, 353.1603317260742, 354.70189452171326, 352.4627106189728, 352.9726560115814, 353.3002257347107, 336.5402991771698, 354.7222361564636, 352.6233661174774, 354.1576383113861, 353.0054614543915, 353.2145848274231]
Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688, 357.24955701828003, 358.9602324962616, 358.5004668235779, 358.01245880126953, 358.2048010826111, 

Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688, 357.24955701828003, 358.9602324962616, 358.5004668235779, 358.01245880126953, 358.2048010826111, 353.1637215614319, 357.5835726261139, 359.3443167209625, 347.3865683078766, 352.90055751800537, 353.96637988090515, 354.0449049472809, 354.72680401802063, 354.8077161312103, 350.10274028778076, 354.1178276538849, 353.5008897781372, 355.3994915485382, 353.38027358055115, 354.62901973724365, 353.0570468902588, 353.48124504089355, 353.03538751602173, 349.43913555145264, 354.10142970085144, 353.2013273239136, 354.0940809249878, 353.1603317260742, 354.70189452171326, 352.4627106189728, 352.9726560115814, 353.3002257347107, 336.5402991771698, 354.7222361564636, 352.6233661174774, 354.1576383113861, 353.0054614543915, 353.2145848274231, 353.124409198761, 353.2495882511139, 353.218359708786, 349.57540583610535, 352.8668622970581, 352.4646375179291, 353.01346588134766, 353.73080921173096, 352.39928936958313, 353.3364

Insert [374.0993812084198, 354.6427631378174, 361.3451943397522, 360.8965365886688, 357.24955701828003, 358.9602324962616, 358.5004668235779, 358.01245880126953, 358.2048010826111, 353.1637215614319, 357.5835726261139, 359.3443167209625, 347.3865683078766, 352.90055751800537, 353.96637988090515, 354.0449049472809, 354.72680401802063, 354.8077161312103, 350.10274028778076, 354.1178276538849, 353.5008897781372, 355.3994915485382, 353.38027358055115, 354.62901973724365, 353.0570468902588, 353.48124504089355, 353.03538751602173, 349.43913555145264, 354.10142970085144, 353.2013273239136, 354.0940809249878, 353.1603317260742, 354.70189452171326, 352.4627106189728, 352.9726560115814, 353.3002257347107, 336.5402991771698, 354.7222361564636, 352.6233661174774, 354.1576383113861, 353.0054614543915, 353.2145848274231, 353.124409198761, 353.2495882511139, 353.218359708786, 349.57540583610535, 352.8668622970581, 352.4646375179291, 353.01346588134766, 353.73080921173096, 352.39928936958313, 353.3364

Update [352.7802982330322, 352.437052488327, 347.89931416511536, 352.3517143726349, 353.0266983509064, 351.8179838657379, 352.7444758415222, 352.07892537117004, 353.1341230869293, 342.2987656593323, 343.8891360759735, 351.3723204135895, 352.31070375442505, 352.2343056201935, 352.45879197120667, 352.41235184669495, 349.92908120155334, 352.17070269584656, 352.34708428382874, 353.90777254104614, 351.8786895275116, 352.9864625930786, 351.97003078460693, 353.8665246963501, 350.41287088394165, 353.04676508903503, 351.83990240097046, 353.2473270893097]
Update [352.7802982330322, 352.437052488327, 347.89931416511536, 352.3517143726349, 353.0266983509064, 351.8179838657379, 352.7444758415222, 352.07892537117004, 353.1341230869293, 342.2987656593323, 343.8891360759735, 351.3723204135895, 352.31070375442505, 352.2343056201935, 352.45879197120667, 352.41235184669495, 349.92908120155334, 352.17070269584656, 352.34708428382874, 353.90777254104614, 351.8786895275116, 352.9864625930786, 351.9700307846

KeyboardInterrupt: 

## Insert Times

In [ ]:
print(insertTimes)

In [ ]:
plt.bar(categories, insertTimes)
plt.plot(categories, insertTimes, color='red')
plt.show()

## Update Times

In [ ]:
print(updateTimes)

In [ ]:
plt.bar(categories, updateTimes)
plt.plot(categories, updateTimes, color='red')
plt.show()

## Select Times

In [ ]:
print(selectTimes)

In [ ]:
plt.bar(categories, selectTimes)
plt.plot(categories, selectTimes, color='red')
plt.show()